In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df=pd.read_csv('/content/drive/MyDrive/Self Study/data.csv')
df.head()

,id,od,le,lc,turn,fluid,fr,orientation,heatw,rth,type
0,2.0,4.0,15.0,40.0,8,1,70,60.0,0.111111,2.094579,Tabular
1,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112346,1.437566,Tabular
2,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112359,1.198448,Tabular
3,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112360,1.146975,Tabular
4,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112360,0.976089,Tabular


In [3]:
df.isnull().sum()

id             0
od             0
le             0
lc             0
turn           0
fluid          0
fr             0
orientation    0
heatw          0
rth            0
type           0
dtype: int64

In [4]:
df = pd.get_dummies(df, drop_first=True) 
df.head()

,id,od,le,lc,turn,fluid,fr,orientation,heatw,rth,type_Tabular
0,2.0,4.0,15.0,40.0,8,1,70,60.0,0.111111,2.094579,1
1,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112346,1.437566,1
2,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112359,1.198448,1
3,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112360,1.146975,1
4,2.0,4.0,15.0,40.0,8,1,70,60.0,0.112360,0.976089,1


In [5]:
X = df.drop(('rth'), axis=1)
y = df['rth']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rfr = RandomForestRegressor()

In [7]:
# Trees in random forest
n_estimators = [int(x) for x in np.linspace(start=100, stop=1200, num=12)]

# Features to consider at every split
max_features = ['auto', 'sqrt']

# Max number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num=6)]
# max_depth.append(None)

# Min number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]

# Min number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [8]:
# Creating the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


In [9]:
rf_random = RandomizedSearchCV(estimator=rfr, 
                               param_distributions=random_grid, 
                               scoring='neg_mean_squared_error', 
                               n_iter=10, cv=5, verbose=2, 
                               random_state=42, n_jobs=1)

In [10]:
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   1.0s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.8s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=   1.0s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=   1.7s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimator

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [11]:
y_pred = rf_random.predict(X_test)

In [12]:
print('MAE: ', metrics.mean_absolute_error(y_test, y_pred))
print('MSE: ', metrics.mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE:  0.18401315393735126
MSE:  0.2383977480716475
RMSE:  0.4882599185594159


In [13]:
score = metrics.r2_score(y_test, y_pred) * 100
print('Score:', score, '%' )

Score: 99.84410290766975 %
